# Are there concerns about what instructors are using?
* Are there unlisenced products instructors are using?
* Who is using New Box View? 
* Force completion final exams? 

In [2]:
from sqlalchemy import create_engine
from os import getenv as ge
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
bbuser, bbpass, bbhost = ge('bbuser'), ge('bbpass'), ge('bbhost')
con_string = f'oracle+cx_oracle://{bbuser}:{bbpass}@{bbhost}'
engine = create_engine(con_string, max_identifier_length=128)

## Which courses are using YouTube?

In [6]:
query = """
SELECT SUBSTR(cm.COURSE_ID,0,4) term, count(cm.COURSE_ID) num_courses
FROM BB_BB60.COURSE_MAIN cm
JOIN (select crsmain_pk1, count(pk1) as youtube
           from bb_bb60.course_contents
           where cnthndlr_handle = 'resource/x-bb-youtube-mashup'
           group by crsmain_pk1) youtube on youtube.crsmain_pk1 = cm.pk1
WHERE cm.COURSE_ID LIKE '42%'
GROUP BY SUBSTR(cm.COURSE_ID,0,4)
ORDER BY term
"""
pd.read_sql(query, con=engine).head()

,term,num_courses
0,4202,301
1,4206,95
2,4209,21


In [13]:
query = """
SELECT cm.COURSE_ID, count(youtube) youtube_count
FROM BB_BB60.COURSE_MAIN cm
JOIN (select crsmain_pk1, count(pk1) as youtube
           from bb_bb60.course_contents
           where cnthndlr_handle = 'resource/x-bb-youtube-mashup'
           group by crsmain_pk1) youtube on youtube.crsmain_pk1 = cm.pk1
WHERE cm.COURSE_ID LIKE '4206-%'
GROUP BY cm.COURSE_ID
ORDER BY youtube_count desc
"""
pd.read_sql(query, con=engine).head()

,course_id,youtube_count
0,4206-74310,1
1,4206-77659,1
2,4206-73407,1
3,4206-75321,1
4,4206-75609,1


## Which courses are using monitor?


In [14]:
query = """
SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       cc.CNTHNDLR_HANDLE,
       cc.DTCREATED,
       cc.DTMODIFIED

    FROM BB_BB60.COURSE_CONTENTS cc
    LEFT JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    LEFT JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1

    WHERE cm.COURSE_ID LIKE '4202-%' and
          cc.CNTHNDLR_HANDLE = 'resource/x-bb-asmt-test-link' and
          cc.TITLE LIKE '%espondus%'
"""
pd.read_sql(query, con=engine).head()

,course_id,course_name,due_date,pk1,title,available_ind,cnthndlr_handle,dtcreated,dtmodified
0,4202-41593,2020Spring-ABSC 410 Behavioral Approaches in W...,2020-04-08 23:59:00,8207683,Exam II- Requires Respondus LockDown Browser,N,resource/x-bb-asmt-test-link,2020-03-31 10:39:19,2020-05-12 23:05:55
1,4202-41593,2020Spring-ABSC 410 Behavioral Approaches in W...,2020-05-13 23:59:00,8279244,Final Exam- Requires Respondus LockDown Browser,Y,resource/x-bb-asmt-test-link,2020-05-05 15:55:55,2020-05-20 10:12:16
2,4202-41593,2020Spring-ABSC 410 Behavioral Approaches in W...,NaT,8220593,PRACTICE EXAM- Requires Respondus LockDown Bro...,Y,resource/x-bb-asmt-test-link,2020-04-06 16:59:34,2020-05-05 15:58:27
3,4202-44571,2020Spring-NAVY 300 Navigation and Operations ...,2020-04-02 23:59:00,8212248,Exam 2 (**Webcam**) - Requires Respondus LockD...,Y,resource/x-bb-asmt-test-link,2020-04-01 19:53:51,2020-04-03 09:07:19
4,4202-44571,2020Spring-NAVY 300 Navigation and Operations ...,2020-04-22 23:59:00,8243477,Exam 3 (**Webcam**) - Requires Respondus LockD...,Y,resource/x-bb-asmt-test-link,2020-04-17 13:20:27,2020-04-20 10:33:32


### Problems
* Included lockdown browser
* No clear actionable information

In [12]:
query = """
SELECT distinct u.USER_ID, 
                u.email, 
                u.firstname || ' ' || u.lastname name

    FROM BB_BB60.COURSE_CONTENTS cc
    LEFT JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    LEFT JOIN BB_BB60.COURSE_USERS cu on cm.PK1 = cu.CRSMAIN_PK1
    LEFT JOIN BB_BB60.USERS u on cu.USERS_PK1 = u.PK1
    WHERE cm.COURSE_ID LIKE '4202-%' and
          cc.CNTHNDLR_HANDLE = 'resource/x-bb-asmt-test-link' and
          cc.TITLE LIKE '%**Webcam**%' and
          (cu.ROLE = 'T' or cu.ROLE = 'P')
"""
pd.read_sql(query, con=engine).head()

,user_id,email,name
0,d643b894,douglas.brown@ku.edu,Douglas Brown
1,t835b558,tab@ku.edu,Thomas Butler
2,m426d149,michaeldeer@ku.edu,Michael Deer
3,m663b326,michael_bell@ku.edu,Michael Bell
4,e854m324,lizannmoddelmog@ku.edu,Liz Ann Moddelmog


## Who is using Force Completion?

In [16]:
query = """
SELECT cm.COURSE_ID,
       cm.COURSE_NAME,
       gm.DUE_DATE,
       cc.PK1,
       cc.TITLE,
       cc.AVAILABLE_IND,
       gm.POSSIBLE,
       cc.CNTHNDLR_HANDLE,
       cc.DTCREATED,
       cc.DTMODIFIED,
       gm.MULTIPLE_ATTEMPTS,
       ca.FORCE_COMPLETION_IND,
       ca.time_limit

    FROM BB_BB60.COURSE_CONTENTS cc
    LEFT JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    LEFT JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.QTI_ASI_DATA qti on gm.QTI_ASI_DATA_PK1 = qti.PK1
    LEFT JOIN BB_BB60.COURSE_ASSESSMENT ca on qti.PK1 = ca.QTI_ASI_DATA_PK1

    WHERE cm.COURSE_ID LIKE '4202-%' and
          cc.CNTHNDLR_HANDLE = 'resource/x-bb-asmt-test-link' and
          ca.FORCE_COMPLETION_IND = 'Y'
"""
pd.read_sql(query, con=engine).head()

,course_id,course_name,due_date,pk1,title,available_ind,possible,cnthndlr_handle,dtcreated,dtmodified,multiple_attempts,force_completion_ind,time_limit
0,4202-52721,2020Spring-HEIM 415 Healthcare Delivery System...,2020-04-26 23:59:00,7948878,QUIZ 12,Y,10.0,resource/x-bb-asmt-test-link,2020-01-06 18:07:46,2020-01-18 12:02:14,1,Y,7.0
1,4202-52721,2020Spring-HEIM 415 Healthcare Delivery System...,2020-02-02 23:59:00,7948830,QUIZ 2,Y,10.0,resource/x-bb-asmt-test-link,2020-01-06 18:07:44,2020-01-18 11:58:01,1,Y,7.0
2,4202-52721,2020Spring-HEIM 415 Healthcare Delivery System...,2020-02-09 23:59:00,7948831,QUIZ 3,Y,10.0,resource/x-bb-asmt-test-link,2020-01-06 18:07:44,2020-01-18 11:58:44,1,Y,7.0
3,4202-52721,2020Spring-HEIM 415 Healthcare Delivery System...,2020-02-23 23:59:00,7948843,QUIZ 5,Y,10.0,resource/x-bb-asmt-test-link,2020-01-06 18:07:45,2020-01-18 11:59:37,1,Y,7.0
4,4202-52721,2020Spring-HEIM 415 Healthcare Delivery System...,2020-03-01 23:59:00,7948844,QUIZ 6,Y,10.0,resource/x-bb-asmt-test-link,2020-01-06 18:07:45,2020-01-18 12:00:04,1,Y,7.0


In [10]:
query = """
SELECT distinct u.USER_ID, 
                u.email, 
                u.firstname || ' ' || u.lastname name

    FROM BB_BB60.COURSE_CONTENTS cc
    LEFT JOIN BB_BB60.COURSE_MAIN cm on cc.CRSMAIN_PK1 = cm.PK1
    LEFT JOIN BB_BB60.GRADEBOOK_MAIN gm on cc.PK1 = gm.COURSE_CONTENTS_PK1
    LEFT JOIN BB_BB60.QTI_ASI_DATA qti on gm.QTI_ASI_DATA_PK1 = qti.PK1
    LEFT JOIN BB_BB60.COURSE_ASSESSMENT ca on qti.PK1 = ca.QTI_ASI_DATA_PK1
    LEFT JOIN BB_BB60.COURSE_USERS cu on cm.PK1 = cu.CRSMAIN_PK1
    LEFT JOIN BB_BB60.USERS u on cu.USERS_PK1 = u.PK1
    
    WHERE cm.COURSE_ID LIKE '4206-%' and
          cc.CNTHNDLR_HANDLE = 'resource/x-bb-asmt-test-link' and
          ca.FORCE_COMPLETION_IND = 'Y' and
          (cu.ROLE = 'T' or cu.ROLE = 'P')
"""
pd.read_sql(query, con=engine).head()

,user_id,email,name
0,shlayman,shlayman@ku.edu,Shannon Layman
1,b084g452,bsgordon@ku.edu,Brian Gordon
2,lmramba,lmramba@ku.edu,Lazarus Mramba
3,summer,summer@ku.edu,Summer Peixoto
4,e696s865,e696s865@kumc.edu,Elena Shergina
